In [1]:
import os
import sys
import math
import operator
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from keras.layers import Dense
from keras.layers import Conv2D
from multiprocessing import Pool
from keras.utils import Sequence
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential
from collections import OrderedDict
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import load_model, save_model
os.chdir("../../")

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class ImageDataGenerator(Sequence):
    
    def __init__(self, x_metadata, y_metadata, batch_size, crop_size):
        self.x = x_metadata
        self.y = y_metadata
        self.batch_size = batch_size
        self.cp = crop_size
    
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array([np.transpose(np.array(tif.imread(file_name), dtype=int)/255.0,
                                     (1,2,0))[self.cp:-self.cp,self.cp:-self.cp,:]
                        for file_name in batch_x]), np.array(batch_y)         

class CNN_Model:
    
    def __init__(self, directory):
        
        self.path = directory
        df = pd.read_csv("occurrences_train.csv",low_memory=False)
        with open("Data/hierarchy_data.pkl","rb") as f:
            hd = pkl.load(f)
        with open("Data/class_encoding.pkl","rb") as f:
            self.classes = pkl.load(f)
        with open("Data/order_encoding.pkl","rb") as f:
            self.orders = pkl.load(f)
        with open("Data/family_encoding.pkl","rb") as f:
            self.families = pkl.load(f)
        with open("Data/genus_encoding.pkl","rb") as f:
            self.genuses = pkl.load(f)
        with open("Data/specie_encoding.pkl","rb") as f:
            self.species = pkl.load(f)

        self.train_pathdata_x = []
        self.train_seq_y = []
        self.test_pathdata_x = []
        self.test_seq_y = []
        
        for cls in hd.keys():
            for order in hd[cls].keys():
                for family in hd[cls][order].keys():
                    for genus in hd[cls][order][family].keys():
                        for specie in hd[cls][order][family][genus]:
                            for im in os.listdir(self.path+"train/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                 +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
                                self.test_pathdata_x.append(self.path+"train/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                             +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)+"/"
                                                             +im)
                            
        for cls in hd.keys():
            for order in hd[cls].keys():
                for family in hd[cls][order].keys():
                    for genus in hd[cls][order][family].keys():
                        for specie in hd[cls][order][family][genus]:
                            for im in os.listdir(self.path+"test/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                 +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)):
                                self.test_pathdata_x.append(self.path+"test/"+str(self.classes[cls])+"/"+str(self.orders[order])
                                                             +"/"+str(self.families[family])+"/"+str(self.genuses[genus])+"/"+str(specie)+"/"
                                                             +im)
        
        np.random.shuffle(self.train_pathdata_x)
        np.random.shuffle(self.test_pathdata_x)
        
        for p in self.train_pathdata_x:
            y = p.split("/")
            c = int(y[3])
            o = int(y[4])
            f = int(y[5])
            g = int(y[6])
            s = int(y[7])
            self.train_seq_y.append(np.array([[c],[o],[f],[g],[s]]))
            
        for p in self.test_pathdata_x:
            y = p.split("/")
            c = int(y[3])
            o = int(y[4])
            f = int(y[5])
            g = int(y[6])
            s = int(y[7])
            self.train_seq_y.append(np.array([[c],[o],[f],[g],[s]]))
            
    def model_create(self):
        
        classifier = Sequential()
        # Step 1 - Convolution
        classifier.add(TimeDistributed(Conv2D(filters=96, kernel_size=(2, 2), activation = 'relu'), input_shape=(5,32,32,33)))
        classifier.add(TimeDistributed(Conv2D(filters=256, kernel_size=(2, 2), activation = 'relu')))
        classifier.add(TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), activation = 'relu')))
        classifier.add(TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), activation = 'relu')))
        #classifier.add(Conv2D(filters=, kernel_size=(2, 2), activation = 'relu'))
        classifier.add(TimeDistributed(MaxPool2D(pool_size = (2, 2))))
        # Step 3 - Flattening
        classifier.add(TimeDistributed(Flatten()))
        # Step 4 - LSTM
        classifier.add(LSTM(100, return_sequences=True))
        # Step 5 - Full connection
        classifier.add(Dense(1, activation = 'relu'))
        # Compiling the CNN
        classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        classifier.summary()
        return classifier
    
    def fit_generator(self, num_epochs=10, batch_size=32, crop_size=16):        
        try:
            classifier = load_model("Code/Models/CNN-RNN_1.h5")
        except:
            classifier = self.model_create()
            train_data = ImageDataGenerator(self.train_pathdata_x, self.train_seq_y, batch_size, crop_size)
            history = classifier.fit_generator(train_data, epochs=num_epochs, use_multiprocessing=True,shuffle=True)
            classifier.save("Code/Models/CNN-RNN_1.h5")
        train_data = ImageDataGenerator(self.test_pathdata_x, self.test_seq_y, batch_size, crop_size)
        scores = classifier.evaluate_generator(test_data, use_multiprocessing=True)
        print("Loss : ", scores[0])
        print("Accuracy : ", scores[1])

In [4]:
ob = CNN_Model("Data/Hierarchial Data/")

In [5]:
ob.fit_generator()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 5, 31, 31, 96)     12768     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 30, 30, 256)    98560     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 28, 28, 256)    590080    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 26, 26, 512)    1180160   
_________________________________________________________________
time_distributed_5 (TimeDist (None, 5, 13, 13, 512)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 5, 86528)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 100)            34651600  
__________

AttributeError: 'ProgbarLogger' object has no attribute 'log_values'